## Real Time Testing

In [12]:
# ! py -3.9 -m pip install cvzone==1.5.6
# ! py -3.9 -m pip install opencv-python==4.7.0.72
# ! py -3.9 -m pip install numpy==1.26.2
# ! py -3.9 -m pip install mediapipe==0.9.1.0
# ! py -3.9 -m pip install tensorflow==2.11.0
# ! py -3.9 -m pip install wordninja==2.0.0
# ! py -3.9 -m pip install transformers==4.35.2
# ! py -3.9 -m pip install kivy==2.1.0
# ! py -3.9 -m pip install kivymd==1.1.1
# ! py -3.9 -m pip install gtts
# ! py -3.9 -m pip install deep-translator
# ! py -3.9 -m pip install pyttsx3
# ! py -3.9 -m pip install playsound
# ! py -3.9 -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# ! py -3.9 -m pip list

In [13]:
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from time import time
from cvzone import FPS

import cv2
import numpy as np
from itertools import chain
from time import time

from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

In [14]:
## 2. Feature Extraction (Hand+Face+Pose Detection)
# Flatten a 2d np array into 1d array
def flatten2dList(arr, dataType=int):
    return np.fromiter(chain.from_iterable(arr), dataType)

# Get the largest absolute value in an np array
def getAbsLargestVal(arr):
    return np.max(np.abs(arr))

# Offset and normalize the landmark list
# Returns a 1d numpy array
def preprocess_landmarks(landmark_list):    
    landmark_list = np.array(landmark_list, dtype=float)
    origin = landmark_list[0]
    
    # Offset every point with respect to the first point
    # Convert to 1D-array
    new_landmark_list = (landmark_list - origin).ravel()
    
    # Get highest absolute value
    largest_value = getAbsLargestVal(new_landmark_list)
    
    # Normalization
    if largest_value != 0:
        return new_landmark_list / largest_value
    return new_landmark_list

# Offset and normalize a BBOX list (BBOX = Bounding Box, used in face and hand detection)
# Returns a 1d numpy array
def preprocess_bbox(bbox, frameSize):
    bbox = np.array(bbox, dtype=float)
    # Convert 3rd and 4th element into coordinates instead of width/height
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]

    # Normalize against frame size
    bbox[0] /= frameSize[0]
    bbox[1] /= frameSize[1]
    bbox[2] /= frameSize[0]
    bbox[3] /= frameSize[1]

    return bbox

# Normalize a center vertex (a list of 2 elements)
# Returns a 1d numpy array
def preprocess_center(center, frameSize):
    center = np.array(center, dtype=float)
    center[0] /= frameSize[0]
    center[1] /= frameSize[1]
    return center

# Preprocess (Offset and normalize) the body's landmark list, bbox and center
def preprocess_body_part(bodyPart, frameSize):
    bodyPart['lmList'] = preprocess_landmarks(bodyPart['lmList'])
    bodyPart['bbox'] = preprocess_bbox(bodyPart['bbox'], frameSize)
    bodyPart['center'] = preprocess_center(bodyPart['center'], frameSize)
    return bodyPart

# Function to generate empty/placeholder data for a hand 
# Used when a hand is not detected in frame
def generate_empty_hand(type):
    return {
        'lmList': np.zeros(21 * 3, dtype=int), 
        'bbox': np.zeros(4, dtype=float), 
        'center': np.zeros(2, dtype=float), 
        'type': type
    }

# Select the best matching face, aka the one with the best score (clarity)
# and closest to the center of the screen
# Since the Neural network will be design to only accept one face
def select_best_matching_face(faces, frameSize):
    if not faces or len(faces) == 0:
        return False
    elif len(faces) == 1:
        return faces[0]
    
    def difference(a, b):
        return ((a[0] - b[0])**2) + ((a[1] - b[1])**2)
    
    frameCenter = (frameSize[0] / 2, frameSize[1] / 2)

    best_score = faces[0]
    best_center = faces[0]
    center_diff = difference(faces[0]['center'], frameCenter)

    for each in faces:
        if difference(each['center'], frameCenter) < center_diff:
            best_center = each
        if each['score'][0] > best_score['score'][0]:
            best_score = each
    
    if best_center['score'][0] > 0.5:
        return best_center
    return best_score

# Flatten everything
def flattenDetectionResult(obj):
    # return np.fromiter(chain.from_iterable([obj['lmList'], obj['bbox'], obj['center']]), float)
    return np.concatenate([obj['lmList'], obj['bbox'], obj['center']])

In [15]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def featureExtractionV3(handDetector, faceDetector, poseDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    # Pose Detection
    # **We only use the first 23 out of the total 33 landmark points 
    #   as those represent the lower half body and are irrelevant to sign language interpretation
    def detectPose(poseDetector, frame, draw):
        frame = poseDetector.findPose(frame, draw=draw)
        results, _ = poseDetector.findPosition(frame, bboxWithHands=False)
        if results:
            results = preprocess_landmarks(results[:23])
        else:
            results = np.zeros(23, dtype=int)
        return results
    
    # Face Detection
    def detectFace(faceDetector, frame, frameSize, draw):
        frame, results = faceDetector.findFaces(frame, draw=draw)
        if results:
            results = select_best_matching_face(results, frameSize)
            results['bbox'] = preprocess_bbox(results['bbox'], frameSize)
            results['center'] = preprocess_center(results['center'], frameSize)
        else:
            results = {
                'bbox': np.zeros(4, dtype=float), 
                'center': np.zeros(2, dtype=float)
            }
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        t2 = executor.submit(detectPose, poseDetector, frame, draw)
        t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            t2.result(), 
            t3.result()['bbox'],
            t3.result()['center'],
            t3.result()['center'] - t1.result()[0]['center'],
            t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [16]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def staticFeatureExtraction(handDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        # t2 = executor.submit(detectPose, poseDetector, frame, draw)
        # t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            # t2.result(), 
            # t3.result()['bbox'],
            # t3.result()['center'],
            # t3.result()['center'] - t1.result()[0]['center'],
            # t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [24]:
import keras
from files_io import readActionLabels

# Load the model from the H5 file
model = keras.models.load_model('../static_recognition/models/static_model_ANN.h5')
# model = keras.models.load_model('../static_recognition/models/static_model_V1.h5')
# model = keras.models.load_model('../static_recognition/models/static_model.keras')

# Print the summary before loading
model.summary()

static_labels = readActionLabels()
static_labels

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 138)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8896      
                                                                 
 dense_11 (Dense)            (None, 128)               8320      
                                                                 
 batch_normalization_3 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 36)                4644      
                                                      

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10']

In [27]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Detectors
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)
fpsReader = FPS()

timeStats = []

try:
    predictionHistory = deque()
    detectionThreshold = 0.999    
    predictionCooldown = 0.5
    appendCooldown = 1.0
    # the first time append should eliminate predictionCooldown
    lastAppendTime = time() + appendCooldown
    lastDetectTime = time() + predictionCooldown

    while True:
        startTime = time()

        # Read from camera
        success, frame = cam.read()
        if not success:
            raise Exception("No Frames Read")
        frame = cv2.flip(frame, 1)

        # Hand Detection
        inputValue, frame = staticFeatureExtraction(
            handDetector, frame)
        
        # detectionResults, frame = featureExtractionV3(
        #     handDetector, faceDetector, poseDetector, frame)
        
        detectionResults = np.expand_dims(
            inputValue, axis=0) # reshape detection result
        
        if time() <= lastDetectTime + predictionCooldown:
            pass
        else:
            predictionResults = model.predict(
                x=detectionResults,
                verbose=0,
                use_multiprocessing=True,
                workers=4
            )[0]
            # lastDetectTime = time()
            
            print(predictionResults)
            
            # Get predicted character and accuracy
            predCharacter = static_labels[np.argmax(predictionResults)]
            predAccuracy = predictionResults[np.argmax(predictionResults)]
        
            cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)          
            
            if predAccuracy >= detectionThreshold:            
                # If predictionHistory is not empty
                # If predCharacter is the same as the last appended character
                # Check if 0.5 seconds have passed since the last append
                if predictionHistory and predCharacter == predictionHistory[-1] and time() <= lastAppendTime + appendCooldown:
                    # Do nothing, don't append
                    pass
                else:
                    predictionHistory.append(predCharacter)
                    # Reset the timestamp when a new character is detected
                    lastAppendTime = time()

                if len(predictionHistory) > 5:
                        predictionHistory.popleft()  
                        
        # cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
        #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        
        cv2.putText(frame, str(predictionHistory), (15, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        

        fps, frame = fpsReader.update(frame, pos=(
            950, 80), color=(0, 255, 0),scale=5, thickness=5)

        # fps, frame = fpsReader.update(frame, pos=(
        #     950, 80), color=(0, 255, 0), scale=5, thickness=5)

        
        # Show resulting frame
        # cv2.putText(frame, f'Training #{training + 1} for \'{action}\'', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow("Sign Language Recognition Prototype", frame)

        timeStats.append(time() - startTime)

        keyPressed = cv2.waitKey(1)
        # Stop Program when pressed 'Esc'
        if keyPressed == 27:
            raise Exception("Finished")
        elif keyPressed == ord('c'):
            predictionHistory.clear()
            lastAppendTime = time() + predictionCooldown


except Exception as e:
    cam.release()
    cv2.destroyAllWindows()
    print(e)

cam.release()
cv2.destroyAllWindows()

[9.76208892e-09 4.81652718e-09 8.85256332e-13 4.48217719e-07
 5.61088349e-08 1.89594144e-03 7.72062592e-08 9.93611753e-01
 3.61865896e-05 1.00341867e-10 6.72348779e-06 4.63540528e-06
 1.11034984e-04 6.73986460e-06 3.57350807e-08 2.25299201e-03
 7.39411262e-05 3.03833957e-07 1.64004543e-08 2.57584674e-04
 1.09214860e-03 3.65021748e-07 5.55915585e-06 1.71273405e-05
 2.54809081e-08 1.08431159e-05 5.15749154e-04 1.93521217e-08
 2.85313206e-07 1.03645569e-07 3.70719761e-07 9.30406095e-05
 3.64774255e-09 2.32244055e-08 5.11635972e-06 8.24389133e-07]
[9.76208892e-09 4.81652718e-09 8.85256332e-13 4.48217719e-07
 5.61088349e-08 1.89594144e-03 7.72062592e-08 9.93611753e-01
 3.61865896e-05 1.00341867e-10 6.72348779e-06 4.63540528e-06
 1.11034984e-04 6.73986460e-06 3.57350807e-08 2.25299201e-03
 7.39411262e-05 3.03833957e-07 1.64004543e-08 2.57584674e-04
 1.09214860e-03 3.65021748e-07 5.55915585e-06 1.71273405e-05
 2.54809081e-08 1.08431159e-05 5.15749154e-04 1.93521217e-08
 2.85313206e-07 1.03645

In [26]:
timeStats[10:]

[0.04279804229736328,
 0.09335613250732422,
 0.11097908020019531,
 0.09581756591796875,
 0.0919039249420166,
 0.1653149127960205,
 0.03846478462219238,
 0.09580421447753906,
 0.0944054126739502,
 0.11049723625183105,
 0.09586262702941895,
 0.14478635787963867,
 0.06071209907531738,
 0.09216666221618652,
 0.09479713439941406,
 0.11281561851501465,
 0.09471940994262695,
 0.14505481719970703,
 0.03985738754272461,
 0.11158108711242676,
 0.09376072883605957,
 0.09618401527404785,
 0.10657024383544922,
 0.14861369132995605,
 0.0411376953125,
 0.10714888572692871,
 0.09562039375305176,
 0.09476923942565918,
 0.09560036659240723,
 0.16006255149841309,
 0.04483842849731445,
 0.09372305870056152,
 0.12858891487121582,
 0.09302115440368652,
 0.0938730239868164,
 0.3512098789215088,
 0.049744367599487305,
 0.10906314849853516,
 0.09268593788146973,
 0.09125399589538574,
 0.11200404167175293,
 0.14304447174072266,
 0.049256086349487305,
 0.09543299674987793,
 0.10323929786682129,
 0.09565711021423